<h1><center>Increase Business with Sustainabiity 3.1 </center></h1>

<h1>Data read from excel</h1>

In [1]:
import pandas as pd


In [2]:
df = pd.read_excel('GreenSpot Businesses Raw Data.xlsx')

df['year'] = pd.to_datetime(df['EnrollmentDate']).dt.year
print(df.year.unique())
df.head()

[2010 2013 2015 2008 2009 2012 2011 2018 2016 2014 2017 2019]


,Industry,BusinessName,Id,FirstName,Email,LastName,Address,City,State,Zip,EnrollmentDate,year
0,office_and_education,Clevest Solutions Inc,9641,Jared,jared.brandon@clevest.com,Brandon,#100 - 13911 Wireless Way,Richmond,OH,43215,2010-10-17,2010
1,industrial,Abbott Nutrition,10112,Susan,susan.davis@abbott.com,Davis,1 abbott park way,tipp city,OH,45371,2013-06-24,2013
2,office_and_education,MAPFRE Stadium,10407,Jordan,jenke@columbuscrewsc.com,Enke,1 Black and Gold Blvd,Columbus,OH,43211,2015-04-21,2015
3,restaurant_and_grocery,Cathy Tiffany Salon Lofts,9446,Cathy,cathy@cathytiffany.com,Tiffany,1 E. Campus View Blvd,Columbus,OH,43235,2008-08-02,2008
4,office_and_education,Nationwide Insurance,9728,Kelly,grosenk1@nationwide.com,Grosenbacher,1 Nationwide Plaza,Columbus,OH,43215,2009-10-19,2009


In [186]:
df.Industry.unique()

array(['office_and_education', 'industrial', 'restaurant_and_grocery'],
      dtype=object)

In [3]:
print(len(df))
df_select = df[df['State'] == 'OH'].copy()
print(len(df_select))

1309
1299


In [4]:
df_select['Address'] = df_select.Address.astype(str)
#attern = '|'.join(['#', '.', ','])
df_select['Address'] = df_select.Address.str.replace(',','').str.replace('.','').str.strip()

In [5]:
df_select.head()

,Industry,BusinessName,Id,FirstName,Email,LastName,Address,City,State,Zip,EnrollmentDate,year
0,office_and_education,Clevest Solutions Inc,9641,Jared,jared.brandon@clevest.com,Brandon,#100 - 13911 Wireless Way,Richmond,OH,43215,2010-10-17,2010
1,industrial,Abbott Nutrition,10112,Susan,susan.davis@abbott.com,Davis,1 abbott park way,tipp city,OH,45371,2013-06-24,2013
2,office_and_education,MAPFRE Stadium,10407,Jordan,jenke@columbuscrewsc.com,Enke,1 Black and Gold Blvd,Columbus,OH,43211,2015-04-21,2015
3,restaurant_and_grocery,Cathy Tiffany Salon Lofts,9446,Cathy,cathy@cathytiffany.com,Tiffany,1 E Campus View Blvd,Columbus,OH,43235,2008-08-02,2008
4,office_and_education,Nationwide Insurance,9728,Kelly,grosenk1@nationwide.com,Grosenbacher,1 Nationwide Plaza,Columbus,OH,43215,2009-10-19,2009


<h2>Geocode</h2>

In [6]:
import censusgeocode as cg

In [7]:
def loc2county(row):
    try:
        #print('{},{},{},{}'.format(row['Address'],row['City'],row['State'],row['Zip']))
        str_add = '{},{},{},{}'.format(row['Address'],row['City'],row['State'],row['Zip'])
        #print(str_add)
        result = cg.onelineaddress(str_add)
        
    except (ValueError, KeyError):
        return 'NULL'
    else:
        #print(result)
        if result == []:
            return 'NULL'
        elif 'status' in result[0]['geographies']['2010 Census Blocks'][0]:
            return 'NULL'
        else:
            return result[0]['geographies']['2010 Census Blocks'][0]['STATE']+result[0]['geographies']['2010 Census Blocks'][0]['COUNTY']

In [8]:
import timeit
start = timeit.default_timer()
for index, row in df_select.iterrows():
    #print(index)
    df_select.loc[index,'geoid'] = loc2county(row)
    #break
stop = timeit.default_timer()

print('Time: ', stop - start)  

Time:  4162.541500599999


In [9]:
df_select.head()

,Industry,BusinessName,Id,FirstName,Email,LastName,Address,City,State,Zip,EnrollmentDate,year,geoid
0,office_and_education,Clevest Solutions Inc,9641,Jared,jared.brandon@clevest.com,Brandon,#100 - 13911 Wireless Way,Richmond,OH,43215,2010-10-17,2010,NULL
1,industrial,Abbott Nutrition,10112,Susan,susan.davis@abbott.com,Davis,1 abbott park way,tipp city,OH,45371,2013-06-24,2013,NULL
2,office_and_education,MAPFRE Stadium,10407,Jordan,jenke@columbuscrewsc.com,Enke,1 Black and Gold Blvd,Columbus,OH,43211,2015-04-21,2015,NULL
3,restaurant_and_grocery,Cathy Tiffany Salon Lofts,9446,Cathy,cathy@cathytiffany.com,Tiffany,1 E Campus View Blvd,Columbus,OH,43235,2008-08-02,2008,39049
4,office_and_education,Nationwide Insurance,9728,Kelly,grosenk1@nationwide.com,Grosenbacher,1 Nationwide Plaza,Columbus,OH,43215,2009-10-19,2009,NULL


In [10]:
print(len(df_select))
df_process = df_select[df_select['geoid'] != 'NULL'].copy()
print(len(df_process))

1299
1112


In [11]:
df_process.dtypes

Industry                  object
BusinessName              object
Id                         int64
FirstName                 object
Email                     object
LastName                  object
Address                   object
City                      object
State                     object
Zip                        int64
EnrollmentDate    datetime64[ns]
year                       int64
geoid                     object
dtype: object

In [12]:
df_processed = df_process.loc[df_process['geoid'].isin(['39101','39091', '39159', '39097', '39047', '39041','39117', '39049','39129','39141','39083', '39089','39045', '39127', '39073']),\
                              ['year','geoid','Industry']].copy()
df_processed.loc[:,'count'] = 1    
df_processed.head()

,year,geoid,Industry,count
3,2008,39049,restaurant_and_grocery,1
7,2009,39049,restaurant_and_grocery,1
8,2008,39049,office_and_education,1
9,2008,39049,office_and_education,1
10,2008,39049,office_and_education,1


In [14]:
print(len(df_processed))
df_processed.geoid.unique()


1097


array(['39049', '39041', '39159', '39045', '39073', '39089', '39097',
       '39101'], dtype=object)

In [15]:
#region
grouped_region = df_processed.groupby(['year','Industry'],as_index = False)['count'].sum()
region = grouped_region.pivot(index='year', columns='Industry', values='count').fillna(0)
region.reset_index(inplace = True)
region

Industry,year,industrial,office_and_education,restaurant_and_grocery
0,2008,47,100,45
1,2009,41,66,21
2,2010,21,41,7
3,2011,36,61,9
4,2012,12,50,7
5,2013,15,162,13
6,2014,13,38,12
7,2015,14,64,10
8,2016,13,65,18
9,2017,5,17,5


In [16]:
#county
grouped_county = df_processed.groupby(['geoid','Industry','year'],as_index = False)['count'].sum()
county = grouped_county.pivot_table(index=['geoid','year'], columns='Industry', values='count').fillna(0)
county.reset_index(inplace = True)
county.head()

Industry,geoid,year,industrial,office_and_education,restaurant_and_grocery
0,39041,2008,3.0,3.0,0.0
1,39041,2009,0.0,2.0,0.0
2,39041,2010,2.0,4.0,0.0
3,39041,2011,2.0,1.0,0.0
4,39041,2012,1.0,4.0,1.0
